# US Corporate Analytics - Getting Started

このノートブックでは、US Corporate Analyticsスキルの基本的な使い方を学びます。

## 目次
1. 環境セットアップ
2. 単一企業の財務分析
3. 複数企業の比較
4. ガバナンス分析
5. マクロ経済統合分析
6. レポート生成

## 1. 環境セットアップ

In [ ]:
# 必要なライブラリのインポート
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml

# パスの設定
sys.path.append('..')

from utils.sec_client import SECEdgarClient
from utils.worldbank_client import WorldBankClient
from utils.imf_client import IMFClient
from utils.financial_ratios import FinancialRatioCalculator
from scripts.company_analyzer import CompanyAnalyzer

# 可視化設定
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ ライブラリのインポート完了")

In [ ]:
# 設定ファイルの読み込み
with open('../config/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("✓ 設定ファイルの読み込み完了")
print(f"  - SEC EDGAR API Key: {config['sec_edgar']['api_key'][:20]}...")
print(f"  - User Agent: {config['sec_edgar']['user_agent']}")

## 2. 単一企業の財務分析

Apple Inc. (AAPL) を例に、包括的な財務分析を実施します。

In [ ]:
# 企業分析器の初期化
ticker = "AAPL"
print(f"Analyzing {ticker}...\n")

analyzer = CompanyAnalyzer(ticker, config)

# すべてのデータを取得（過去5年間）
analyzer.fetch_all_data(years=5)

print("\n✓ データ取得完了")

In [ ]:
# 財務データの表示
print("=== 財務データ（過去5年間） ===")
display(analyzer.financial_data)

In [ ]:
# 財務比率の表示
print("=== 財務比率（過去5年間） ===")
display(analyzer.financial_ratios)

In [ ]:
# パフォーマンス分析
analysis = analyzer.analyze_performance()

print(f"\n{'='*60}")
print(f"企業パフォーマンス分析: {ticker}")
print(f"{'='*60}")
print(f"\n分析年度: {analysis['latest_year']}")
print(f"総合評価: {analysis['performance_rating']}\n")

print("主要財務比率（最新年度）:")
for ratio in ['ROA', 'ROE', 'net_margin', 'current_ratio', 'debt_ratio']:
    if ratio in analysis['latest_ratios']:
        value = analysis['latest_ratios'][ratio]
        avg = analysis['average_ratios'][ratio]
        print(f"  {ratio}: {value:.2f} (平均: {avg:.2f})")

In [ ]:
# 可視化: 収益性トレンド
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# ROA
if 'ROA' in analyzer.financial_ratios.columns:
    axes[0, 0].plot(analyzer.financial_ratios.index, 
                    analyzer.financial_ratios['ROA'], 
                    marker='o', linewidth=2, color='#1f77b4')
    axes[0, 0].set_title('ROA Trend', fontsize=14, fontweight='bold')
    axes[0, 0].set_ylabel('ROA (%)')
    axes[0, 0].grid(True, alpha=0.3)

# ROE
if 'ROE' in analyzer.financial_ratios.columns:
    axes[0, 1].plot(analyzer.financial_ratios.index, 
                    analyzer.financial_ratios['ROE'], 
                    marker='s', linewidth=2, color='#ff7f0e')
    axes[0, 1].set_title('ROE Trend', fontsize=14, fontweight='bold')
    axes[0, 1].set_ylabel('ROE (%)')
    axes[0, 1].grid(True, alpha=0.3)

# Net Margin
if 'net_margin' in analyzer.financial_ratios.columns:
    axes[1, 0].plot(analyzer.financial_ratios.index, 
                    analyzer.financial_ratios['net_margin'], 
                    marker='^', linewidth=2, color='#2ca02c')
    axes[1, 0].set_title('Net Margin Trend', fontsize=14, fontweight='bold')
    axes[1, 0].set_ylabel('Net Margin (%)')
    axes[1, 0].grid(True, alpha=0.3)

# Debt Ratio
if 'debt_ratio' in analyzer.financial_ratios.columns:
    axes[1, 1].plot(analyzer.financial_ratios.index, 
                    analyzer.financial_ratios['debt_ratio'], 
                    marker='d', linewidth=2, color='#d62728')
    axes[1, 1].set_title('Debt Ratio Trend', fontsize=14, fontweight='bold')
    axes[1, 1].set_ylabel('Debt Ratio')
    axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'../data/processed/{ticker}_financial_trends.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ グラフを保存: ../data/processed/{ticker}_financial_trends.png")

## 3. 複数企業の比較

テクノロジー業界の主要企業（Apple, Microsoft, Google）を比較します。

In [ ]:
from scripts.industry_comparison import IndustryComparison

# 比較対象企業
tickers = ["AAPL", "MSFT", "GOOGL"]
print(f"Comparing: {', '.join(tickers)}\n")

# 比較分析の実行
comparison = IndustryComparison(tickers, config)
comparison.fetch_all_companies(years=5)

print("\n✓ データ取得完了")

In [ ]:
# 財務比率の比較
comparison_df = comparison.compare_ratios(['ROA', 'ROE', 'net_margin'])
print("=== 財務比率の比較（最新年度） ===")
display(comparison_df)

In [ ]:
# ランキング生成
ranking = comparison.generate_ranking('ROE')
print("=== ROEランキング ===")
display(ranking)

In [ ]:
# 可視化: レーダーチャート
comparison.visualize_comparison(['ROA', 'ROE', 'net_margin', 'current_ratio'])
plt.savefig('../data/processed/tech_companies_comparison.png', dpi=300, bbox_inches='tight')
print("\n✓ グラフを保存: ../data/processed/tech_companies_comparison.png")

## 4. マクロ経済統合分析

企業業績とマクロ経済指標の関係を分析します。

In [ ]:
# マクロ経済データの表示
print("=== マクロ経済データ ===")
display(analyzer.macro_data)

In [ ]:
# 相関分析
from scipy.stats import pearsonr

# データを結合
merged_data = analyzer.financial_ratios.merge(
    analyzer.macro_data,
    left_index=True,
    right_on='year',
    how='inner'
)

print("=== 相関分析 ===")
print("\n企業業績とマクロ経済指標の相関:\n")

# ROA vs GDP成長率
if 'ROA' in merged_data.columns and 'NY.GDP.MKTP.KD.ZG' in merged_data.columns:
    corr, pval = pearsonr(
        merged_data['ROA'].dropna(), 
        merged_data['NY.GDP.MKTP.KD.ZG'].dropna()
    )
    print(f"ROA vs GDP成長率: r={corr:.3f}, p={pval:.4f}")
    if pval < 0.05:
        print("  → 統計的に有意な相関あり **")
    else:
        print("  → 統計的に有意ではない")

## 5. レポート生成

分析結果をレポート形式で出力します。

In [ ]:
# サマリーレポートの生成
report = analyzer.generate_summary_report()
print(report)

In [ ]:
# データのエクスポート
output_dir = "../data/processed"
analyzer.export_data(output_dir)

print(f"\n✓ すべてのデータを {output_dir} にエクスポートしました")
print("\n生成されたファイル:")
print(f"  - {ticker}_financial_data.csv")
print(f"  - {ticker}_financial_ratios.csv")
print(f"  - {ticker}_macro_data.csv")
print(f"  - {ticker}_summary_report.txt")

## まとめ

このノートブックでは、US Corporate Analyticsスキルの基本的な機能を学びました：

1. ✅ SEC EDGARから企業の財務データを取得
2. ✅ 30種類以上の財務比率を自動計算
3. ✅ 複数企業の比較分析
4. ✅ マクロ経済データとの統合分析
5. ✅ 可視化とレポート生成

### 次のステップ

- **02_advanced_analysis.ipynb**: より高度な分析手法
- **03_governance_analysis.ipynb**: コーポレートガバナンス分析
- **04_event_study.ipynb**: イベントスタディ分析

### リソース

- [SKILL.md](../SKILL.md): 詳細なドキュメント
- [README.md](../README.md): プロジェクト概要
- [examples/USAGE_EXAMPLES.md](../examples/USAGE_EXAMPLES.md): 実践的な使用例